In [9]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.utils import np_utils

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import theano
from PIL import Image
from numpy import *
# SKLEARN
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split



In [11]:
import os
from sklearn.utils import shuffle
import glob
import sys
################### DATASET HANDLING ####################
DATASET_PATH = "/home/dic/jupyter/train_Binary" #change the path to your dataset folder here

def parseDataset():
 
    #we use subfolders as class labels
    classes = [folder for folder in sorted(os.listdir(DATASET_PATH))]
 
    #now we enlist all image paths
    images = []
    for c in classes:
        images += ([os.path.join(DATASET_PATH, c, path) for path in os.listdir(os.path.join(DATASET_PATH, c))])
    #print(images)
    #shuffle image paths
    images = shuffle(images, random_state=74)
 
    #we want to use a 15% validation split
    total_len=len(images)
    print("total:",total_len)
    vsplit = int(len(images) * 0.70) #=40
    print("vsplit:",vsplit)
    
    tsplit= int(total_len-vsplit)
    print("tsplit:",tsplit)
    
    gsplit=int(tsplit/2)
    print(gsplit)
    msplit=int(vsplit+gsplit)
    
    
    train = images[:vsplit] #everything except the last vsplit items in the array 
    val = images[vsplit:msplit]  #only last vsplit items from the array
    test=images[msplit:]
    #show some stats
    print ("CLASS LABELS:", classes)
    print ("TRAINING IMAGES:", len(train))
    print ("VALIDATION IMAGES:", len(val))
    print ("TEST IMAGES:", len(test))
    return classes, train, val,test
 
#parse dataset
CLASSES, TRAIN, VAL,TEST = parseDataset()

total: 16567
vsplit: 11596
tsplit: 4971
2485
CLASS LABELS: ['CEP', 'LPV']
TRAINING IMAGES: 11596
VALIDATION IMAGES: 2485
TEST IMAGES: 2486


In [81]:
import cv2
import numpy as np
img_p=[] 
#################### BATCH HANDLING #####################
def loadImageAndTarget(path):
    #print(path)
    #here we open the image and scale it to 64x64 pixels
    img = cv2.imread(path)
    
    #print(path)
    img = cv2.resize(img, (22, 23))
    
    
    #OpenCV uses BGR instead of RGB, but for now we can ignore that
    #our image has the shape (64, 64, 3) but we need it to be (3, 64, 64)
    img = np.transpose(img, (2, 1, 0))
    
    #we want to use subfolders as class labels
    label = path.split(os.sep[-1])[-2]
    #print(label)
 
    #we need to get the index of our label from CLASSES
    index = CLASSES.index(label)
 
    #allocate array for target
    target = np.zeros((2), dtype='float32')
 
    #we set our target array = 1.0 at our label index, all other entries remain zero
    #Example: if label = dog and dog has index 2 in CLASSES, target looks like: [0.0, 0.0, 1.0, 0.0, 0.0]
    target[index] = 1.0
 
    #we need a 4D-vector for our image and a 2D-vector for our targets
    #we can adjust array dimension with reshape
    img = img.reshape(-1, 3, 22, 23)
    target = target.reshape(-1, 2)
    img_p.append(img)
    return img, target
print(img_p)

#a reasonable size for one batch is 128
BATCH_SIZE = 200
def getDatasetChunk(split):
 
    #get batch-sized chunks of image paths
    for i in range(0, len(split), BATCH_SIZE):
        yield split[i:i+BATCH_SIZE]
        
def getNextImageBatch(split):    
 
    #allocate numpy arrays for image data and targets
    #input shape of our ConvNet is (None, 3, 22, 23)
    x_b = np.zeros((BATCH_SIZE, 3, 22, 23), dtype='float32')
    #output shape of our ConvNet is (None, 5) as we have 5 classes
    y_b = np.zeros((BATCH_SIZE, 2), dtype='float32')
 
    #fill batch
    for chunk in getDatasetChunk(split):        
        ib = 0
        for path in chunk:
            #load image data and class label from path
            x, y = loadImageAndTarget(path)
 
            #pack into batch array
            x_b[ib] = x
            y_b[ib] = y
            ib += 1
 
        #instead of return, we use yield
        yield x_b[:len(chunk)], y_b[:len(chunk)]
#x_train= np.array()
x_train=[]
y_train=[]
for image_batch, target_batch in getNextImageBatch(TRAIN):
    x_train.append(image_batch)
#    x_train.append(image_batch)
    y_train.append(target_batch)
x_train = np.array(x_train)  

#print(y_train)        



x_test=[]
y_test=[]
for image_batch, target_batch in getNextImageBatch(TEST):
        x_test.append(image_batch)
        y_test.append(target_batch)


x_val=[]
y_val=[]
for image_batch, target_batch in getNextImageBatch(VAL):
        
        x_val.append(image_batch)
        y_val.append(target_batch)  
        
        


[]


TypeError: 'tuple' object is not callable

In [88]:
import pandas as pd
df_x=pd.DataFrame(x_train) 
print(df_x)

                                                    0
0   [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
1   [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
2   [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
3   [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
4   [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
5   [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
6   [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
7   [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
8   [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
9   [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
10  [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
11  [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
12  [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
13  [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
14  [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
15  [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
16  [[[[128. 128. 128. 128. 128. 128. 128. 128. 12...
17  [[[[128. 128. 128. 128. 

In [ ]:
df_x= data.iloc[:,1:].values.reshape

In [46]:
from lasagne import layers
from lasagne.nonlinearities import softmax, tanh
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
img_rows=22
img_cols=23
num_classes=2
input_shape = (3, img_rows, img_cols) 
################## BUILDING THE MODEL ###################
#def buildModel():

    #this is our input layer with the inputs (None, dimensions, width, height)
    #l_input = layers.InputLayer((None, 3, 22, 23))
model = Sequential()    
    #first convolutional layer, has l_input layer as incoming and is followed by a pooling layer
    #l_conv1 = layers.Conv2DLayer(l_input, num_filters=32, filter_size=3, nonlinearity=lasagne.nonlinearities.tanh)
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    
    #l_pool = layers.MaxPool2DLayer(l_conv1, pool_size=2)
    
    #l_drop1= layers.DropoutLayer(l_conv1,  p=0.1)
model.add(Dropout(0.1))
    #l_dense1 = layers.DenseLayer(l_drop1, num_units=128)
    
    #l_conv2 = layers.Conv2DLayer(l_drop1, num_filters=128, filter_size=5, nonlinearity=lasagne.nonlinearities.rectify)
    
   
    #l_conv3 = layers.Conv2DLayer(l_conv2, num_filters=256, filter_size=5, nonlinearity=lasagne.nonlinearities.rectify)
model.add(Flatten())
    #l_dense2 = layers.DenseLayer(l_drop1, num_units=128)
model.add(Dense(128, activation='relu'))
    
    #l_drop2= layers.DropoutLayer(l_dense2,  p=0.25)
model.add(Dropout(0.25))
    
    #l_dense2 = layers.DenseLayer(l_drop2, num_units=128)
model.add(Dense(128, activation='relu'))
    
    #l_output = layers.DenseLayer(l_dense2, num_units=7, nonlinearity=lasagne.nonlinearities.softmax)
model.add(Dense(num_classes,activation='softmax', name='preds'))
    #let's see how many params our net has
    #print ("MODEL HAS", layers.count_params(l_output), "PARAMS")
    
    #we return the layer stack as our network by returning the last layer
    #return model
#print(model.summary())



In [48]:
#############COMPILING#############
batch_size = 256
epochs = 10
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [49]:

model.fit(x_train, y_train, 
          batch_size=32, nb_epoch=10, verbose=1)

/home/dic/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 58 arrays: [array([[[[128., 128., 128., ..., 128., 128., 128.],
         [128., 128., 128., ..., 128., 128., 128.],
         [128., 128., 128., ..., 128., 128., 128.],
         ...,
         [128.,   0.,   0., ....

In [39]:


X_train = np.asarray(x_train)

X_train = X_train.reshape(X_train.shape[0],1, img_rows, img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_test /= 255
x_val /= 255



ValueError: cannot reshape array of size 58 into shape (58,1,22,23)

In [34]:
###########Training###################

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val))

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 58 arrays: [array([[[[128., 128., 128., ..., 128., 128., 128.],
         [128., 128., 128., ..., 128., 128., 128.],
         [128., 128., 128., ..., 128., 128., 128.],
         ...,
         [128.,   0.,   0., ....